In [ ]:
import keras

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, 
                       logs={}):
       self.log = []
 
    def on_epoch_end(self, 
                     batch, 
                     logs={}):
       self.log.append(logs)

In [ ]:
import math
import inspect

import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin

import keras
from keras.layers import Input, Dense, BatchNormalization, Dropout, convolutional, pooling
from keras.models import Model

import tensorflow

class Convolutional2DAutoencoder(BaseEstimator, 
                                 TransformerMixin):
    def __init__(self, 
                 input_shape=None,
                 n_epoch=None,
                 batch_size=None,
                 encoder_layers=None,
                 decoder_layers=None,
                 filters=None,
                 kernel_size=None,
                 strides=None,
                 pool_size=None,
                 denoising=None):
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")
        
        for arg, val in values.items():
            setattr(self, arg, val)
        
        loss_history = LossHistory()
        
        early_stop = keras.callbacks.EarlyStopping(monitor="val_loss",
                                                   patience=10)
        
        reduce_learn_rate = keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                              factor=0.1,
                                                              patience=20)
        
        self.callbacks_list = [loss_history, early_stop, reduce_learn_rate]

        for i in range(self.encoder_layers):
            if i == 0:
                self.input_data = Input(shape=self.input_shape)
                self.encoded = BatchNormalization()(self.input_data)
                self.encoded = convolutional.Conv2D(filters=self.filters, kernel_size=self.kernel_size, strides=self.strides, activation="elu", padding="same")(self.encoded)
                self.encoded = Dropout(rate=0.5)(self.encoded)
            elif i > 0 and i < self.encoder_layers - 1:
                self.encoded = BatchNormalization()(self.encoded)
                self.encoded = convolutional.Conv2D(filters=self.filters, kernel_size=self.kernel_size, strides=self.strides, activation="elu", padding="same")(self.encoded)
                self.encoded = Dropout(rate=0.5)(self.encoded)
            elif i == self.encoder_layers - 1:
                self.encoded = BatchNormalization()(self.encoded)
                self.encoded = convolutional.Conv2D(filters=self.filters, kernel_size=self.kernel_size, strides=self.strides, activation="elu", padding="same")(self.encoded)

        self.encoded = pooling.MaxPooling2D(strides=self.pool_size, padding="same")(self.encoded)
        self.decoded = BatchNormalization()(self.encoded)
        self.decoded = convolutional.Conv2D(filters=self.filters, kernel_size=self.kernel_size, strides=self.strides, activation="elu", padding="same")(self.decoded)
        self.decoded = convolutional.UpSampling2D(size=self.pool_size)(self.decoded)

        for i in range(self.decoder_layers):
            if i < self.decoder_layers - 1:
                self.decoded = BatchNormalization()(self.decoded)
                self.decoded = convolutional.Conv2D(filters=self.filters, kernel_size=self.kernel_size, strides=self.strides, activation="elu", padding="same")(self.decoded)
                self.decoded = Dropout(rate=0.5)(self.decoded)
            else:
                self.decoded = BatchNormalization()(self.decoded)
                self.decoded = convolutional.Conv2D(filters=self.filters, kernel_size=self.kernel_size, strides=self.strides, activation="elu", padding="same")(self.decoded)

        # 4D tensor with shape: (samples, new_rows, new_cols, filters).
        # Remember think of this as a 2D-Lattice across potentially multiple channels per observation.
        # Rows represent time and columns represent some quantities of interest that evolve over time.
        # Channels might represent different sources of information.
        self.decoded = BatchNormalization()(self.decoded)
        self.decoded = convolutional.Conv2D(filters=self.input_shape[2], kernel_size=self.kernel_size, strides=self.strides, activation="sigmoid", padding="same")(self.decoded)

        self.autoencoder = Model(self.input_data, self.decoded)
        self.autoencoder.compile(optimizer=keras.optimizers.Adam(),
                                 loss="mean_squared_error")
            
    def fit(self,
            X,
            y=None):
        self.autoencoder.fit(X if self.denoising is None else X + self.denoising, X,
                             validation_split=0.3,
                             epochs=self.n_epoch,
                             batch_size=self.batch_size,
                             shuffle=True,
                             callbacks=self.callbacks_list, 
                             verbose=1)

        self.encoded_for_transformer = keras.layers.Flatten()(self.encoded)
        
        self.encoder = Model(self.input_data, self.encoded_for_transformer)
        
        return self
    
    def transform(self,
                  X):
        return self.encoder.predict(X)

In [ ]:
import math
import inspect

import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin

import keras
from keras.layers import Input, Dense, BatchNormalization, Dropout, local, convolutional, Flatten, Reshape
from keras.models import Model

import tensorflow

class ConvolutionalAutoencoder(BaseEstimator, 
                               TransformerMixin):
    def __init__(self, 
                 input_shape=None,
                 n_epoch=None,
                 batch_size=None,
                 encoder_layers=None,
                 decoder_layers=None,
                 filters=None,
                 kernel_size=None,
                 strides=None,
                 pool_size=None,
                 denoising=None):
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")
        
        for arg, val in values.items():
            setattr(self, arg, val)
        
        loss_history = LossHistory()
        
        early_stop = keras.callbacks.EarlyStopping(monitor="val_loss",
                                                   patience=10)
        
        reduce_learn_rate = keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                              factor=0.1,
                                                              patience=20)
        
        self.callbacks_list = [loss_history, early_stop, reduce_learn_rate]

        for i in range(self.encoder_layers):
            if i == 0:
                self.input_data = Input(shape=self.input_shape)
                self.encoded = BatchNormalization()(self.input_data)
                self.encoded = keras.layers.Conv1D(filters=self.filters, kernel_size=self.kernel_size, strides=self.strides, activation="elu", padding="same")(self.encoded)
                self.encoded = Dropout(rate=0.5)(self.encoded)
            elif i > 0 and i < self.encoder_layers - 1:
                self.encoded = BatchNormalization()(self.encoded)
                self.encoded = keras.layers.Conv1D(filters=self.filters, kernel_size=self.kernel_size, strides=self.strides, activation="elu", padding="same")(self.encoded)
                self.encoded = Dropout(rate=0.5)(self.encoded)
            elif i == self.encoder_layers - 1:
                self.encoded = BatchNormalization()(self.encoded)
                self.encoded = keras.layers.Conv1D(filters=self.filters, kernel_size=self.kernel_size, strides=self.strides, activation="elu", padding="same")(self.encoded)

        self.encoded = keras.layers.MaxPooling1D(strides=self.pool_size, padding="valid")(self.encoded)
        self.encoded = BatchNormalization()(self.encoded)
        self.encoded = keras.layers.Conv1D(filters=self.filters, kernel_size=self.kernel_size, strides=self.strides, activation="elu", padding="same")(self.encoded)
        self.decoded = keras.layers.UpSampling1D(size=self.pool_size)(self.encoded)

        for i in range(self.decoder_layers):
            if i < self.decoder_layers - 1:
                self.decoded = BatchNormalization()(self.decoded)
                self.decoded = keras.layers.Conv1D(filters=self.filters, kernel_size=self.kernel_size, strides=self.strides, activation="elu", padding="same")(self.decoded)
                self.decoded = Dropout(rate=0.5)(self.decoded)
            else:
                self.decoded = BatchNormalization()(self.decoded)
                self.decoded = keras.layers.Conv1D(filters=self.filters, kernel_size=self.kernel_size, strides=self.strides, activation="elu", padding="same")(self.decoded)

        # 3D tensor with shape: (batch_size, new_steps, filters).
        # Remember think of this as a 2D-Lattice per observation.
        # Rows represent time and columns represent some quantities of interest that evolve over time.
        self.decoded = BatchNormalization()(self.decoded)
        self.decoded = keras.layers.Conv1D(filters=self.input_shape[1], kernel_size=self.kernel_size, strides=self.strides, activation="sigmoid", padding="same")(self.decoded)

        self.autoencoder = Model(self.input_data, self.decoded)
        self.autoencoder.compile(optimizer=keras.optimizers.Adam(),
                                 loss="mean_squared_error")
            
    def fit(self,
            X,
            y=None):
        self.autoencoder.fit(X if self.denoising is None else X + self.denoising, X,
                             validation_split=0.3,
                             epochs=self.n_epoch,
                             batch_size=self.batch_size,
                             shuffle=True,
                             callbacks=self.callbacks_list, 
                             verbose=1)

        self.encoded_for_transformer = keras.layers.Flatten()(self.encoded)
        
        self.encoder = Model(self.input_data, self.encoded_for_transformer)

        return self
    
    def transform(self,
                  X):
        return self.encoder.predict(X)

In [ ]:
import os
import tensorflow as tf

def get_session(gpu_fraction=0.25,
                allow_soft_placement=True,
                log_device_placement=True):
    """
    https://groups.google.com/forum/#!topic/keras-users/MFUEY9P1sc8
    """
    num_threads = os.environ.get("OMP_NUM_THREADS")
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, intra_op_parallelism_threads=num_threads, allow_soft_placement=allow_soft_placement, log_device_placement=log_device_placement))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, allow_soft_placement=allow_soft_placement, log_device_placement=log_device_placement))

In [ ]:
import math
import inspect

from sklearn.base import BaseEstimator, TransformerMixin

import keras
from keras.layers import Input, Activation, Dense, Dropout, LSTM, RepeatVector, TimeDistributed
from keras.models import Model

import tensorflow

class Seq2SeqAutoencoder(BaseEstimator, 
                         TransformerMixin):
    def __init__(self, 
                 input_shape=None,
                 n_epoch=None,
                 batch_size=None,
                 encoder_layers=None,
                 decoder_layers=None,
                 n_hidden_units=None,
                 encoding_dim=None,
                 stateful=None,
                 denoising=None):
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")
        
        for arg, val in values.items():
            setattr(self, arg, val)
        
        loss_history = LossHistory()
        
        early_stop = keras.callbacks.EarlyStopping(monitor="val_loss",
                                                   patience=10)
        
        reduce_learn_rate = keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                              factor=0.1,
                                                              patience=20)
        
        self.callbacks_list = [loss_history, early_stop, reduce_learn_rate]
        
        # 2D-lattice with time on the x-axis (across rows) and with space on the y-axis (across columns).
        if self.stateful is True:
            self.input_data = Input(batch_shape=self.input_shape)
            self.n_rows = self.input_shape[1]
            self.n_cols = self.input_shape[2]
        else:
            self.input_data = Input(shape=self.input_shape)
            self.n_rows = self.input_shape[0]
            self.n_cols = self.input_shape[1]

        for i in range(self.encoder_layers):
            if i == 0:
                # Returns a sequence of n_rows vectors of dimension n_hidden_units.
                self.encoded = LSTM(units=self.n_hidden_units, return_sequences=True, stateful=self.stateful)(self.input_data)
            else:
                self.encoded = LSTM(units=self.n_hidden_units, return_sequences=True, stateful=self.stateful)(self.encoded)

        # Returns 1 vector of dimension encoding_dim.
        self.encoded = LSTM(units=self.encoding_dim, return_sequences=False, stateful=self.stateful)(self.encoded)

        # Returns a sequence containing n_rows vectors where each vector is of dimension encoding_dim.
        # output_shape: (None, n_rows, encoding_dim).
        self.decoded = RepeatVector(self.n_rows)(self.encoded)

        for i in range(self.decoder_layers):
            self.decoded = LSTM(units=self.n_hidden_units, return_sequences=True, stateful=self.stateful)(self.decoded)
        
        # If return_sequences is True: 3D tensor with shape (batch_size, timesteps, units).
        # Else: 2D tensor with shape (batch_size, units).
        # Note that n_rows here is timesteps and n_cols here is units.
        # If return_state is True: a list of tensors. 
        # The first tensor is the output. The remaining tensors are the last states, each with shape (batch_size, units).
        # If stateful is True: the last state for each sample at index i in a batch will be used as initial state for the sample of index i in the following batch.
        # For LSTM (not LSTM) If unroll is True: the network will be unrolled, else a symbolic loop will be used. Unrolling can speed-up a RNN, although it tends to be more memory-intensive. Unrolling is only suitable for short sequences.
        self.decoded = LSTM(units=self.n_cols, return_sequences=True, stateful=self.stateful)(self.decoded)

        self.autoencoder = Model(self.input_data, self.decoded)
        self.autoencoder.compile(optimizer=keras.optimizers.Adam(),
                                 loss="mean_squared_error")
            
    def fit(self,
            X,
            y=None):
        self.autoencoder.fit(X if self.denoising is None else X + self.denoising, X,
                             validation_split=0.3,
                             epochs=self.n_epoch,
                             batch_size=self.batch_size,
                             shuffle=True,
                             callbacks=self.callbacks_list,
                             verbose=1)

        self.encoder = Model(self.input_data, self.encoded)
        
        return self
    
    def transform(self,
                  X):
        return self.encoder.predict(X)

In [ ]:
import math
import inspect

from sklearn.base import BaseEstimator, TransformerMixin

import keras
from keras.layers import Input, Dense, BatchNormalization, Dropout
from keras.models import Model

import tensorflow


class VanillaAutoencoder(BaseEstimator, 
                         TransformerMixin):
    def __init__(self, 
                 n_feat=None,
                 n_epoch=None,
                 batch_size=None,
                 encoder_layers=None,
                 decoder_layers=None,
                 n_hidden_units=None,
                 encoding_dim=None,
                 denoising=None):
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")
        
        for arg, val in values.items():
            setattr(self, arg, val)
        
        loss_history = LossHistory()
        
        early_stop = keras.callbacks.EarlyStopping(monitor="val_loss",
                                                   patience=10)
        
        reduce_learn_rate = keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                              factor=0.1,
                                                              patience=20)
        
        self.callbacks_list = [loss_history, early_stop, reduce_learn_rate]
        
        for i in range(self.encoder_layers):
            if i == 0:
                self.input_data = Input(shape=(self.n_feat,))
                self.encoded = BatchNormalization()(self.input_data)
                self.encoded = Dense(units=self.n_hidden_units, activation="elu")(self.encoded)
                self.encoded = Dropout(rate=0.5)(self.encoded)
            elif i > 0 and i < self.encoder_layers - 1:
                self.encoded = BatchNormalization()(self.encoded)
                self.encoded = Dense(units=self.n_hidden_units, activation="elu")(self.encoded)
                self.encoded = Dropout(rate=0.5)(self.encoded)
            elif i == self.encoder_layers - 1:
                self.encoded = BatchNormalization()(self.encoded)
                self.encoded = Dense(units=self.n_hidden_units, activation="elu")(self.encoded)
        
        self.encoded = BatchNormalization()(self.encoded)
        self.encoded = Dense(units=self.encoding_dim, activation="sigmoid")(self.encoded)

        for i in range(self.decoder_layers):
            if i == 0:
                self.decoded = BatchNormalization()(self.encoded)
                self.decoded = Dense(units=self.n_hidden_units, activation="elu")(self.decoded)
                self.decoded = Dropout(rate=0.5)(self.decoded)
            elif i > 0 and i < self.decoder_layers - 1:
                self.decoded = BatchNormalization()(self.decoded)
                self.decoded = Dense(units=self.n_hidden_units, activation="elu")(self.decoded)
                self.decoded = Dropout(rate=0.5)(self.decoded)
            elif i == self.decoder_layers - 1:
                self.decoded = BatchNormalization()(self.decoded)
                self.decoded = Dense(units=self.n_hidden_units, activation="elu")(self.decoded)
        
        # Output would have shape: (batch_size, n_feat).
        self.decoded = BatchNormalization()(self.decoded)
        self.decoded = Dense(units=self.n_feat, activation="sigmoid")(self.decoded)

        self.autoencoder = Model(self.input_data, self.decoded)
        self.autoencoder.compile(optimizer=keras.optimizers.Adam(),
                                 loss="mean_squared_error")
           
    def fit(self,
            X,
            y=None):
        self.autoencoder.fit(X if self.denoising is None else X + self.denoising, X,
                             validation_split=0.3,
                             epochs=self.n_epoch,
                             batch_size=self.batch_size,
                             shuffle=True,
                             callbacks=self.callbacks_list, 
                             verbose=1)

        self.encoder = Model(self.input_data, self.encoded)
        
        return self
    
    def transform(self,
                  X):
        return self.encoder.predict(X)

In [ ]:
import math
import inspect

from sklearn.base import BaseEstimator, TransformerMixin

import keras
from keras.layers import Input, Dense, BatchNormalization, Dropout, Lambda, add
from keras.models import Model, Sequential

import tensorflow

class VariationalAutoencoder(BaseEstimator, 
                             TransformerMixin):
    def __init__(self, 
                 n_feat=None,
                 n_epoch=None,
                 batch_size=None,
                 encoder_layers=None,
                 decoder_layers=None,
                 n_hidden_units=None,
                 encoding_dim=None,
                 denoising=None):
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")
        
        for arg, val in values.items():
            setattr(self, arg, val)
        
        loss_history = LossHistory()
        
        early_stop = keras.callbacks.EarlyStopping(monitor="val_loss",
                                                   patience=10)
        
        reduce_learn_rate = keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                              factor=0.1,
                                                              patience=20)
        
        self.callbacks_list = [loss_history, early_stop, reduce_learn_rate]

        for i in range(self.encoder_layers):
            if i == 0:
                self.input_data = Input(shape=(self.n_feat,))
                self.encoded = BatchNormalization()(self.input_data)
                self.encoded = Dense(units=self.n_hidden_units, activation="elu")(self.encoded)
                self.encoded = Dropout(rate=0.5)(self.encoded)
            elif i > 0 and i < self.encoder_layers - 1:
                self.encoded = BatchNormalization()(self.encoded)
                self.encoded = Dense(units=self.n_hidden_units, activation="elu")(self.encoded)
                self.encoded = Dropout(rate=0.5)(self.encoded)
            elif i == self.encoder_layers - 1:
                self.encoded = BatchNormalization()(self.encoded)
                self.encoded = Dense(units=self.n_hidden_units, activation="elu")(self.encoded)
        
        self.mu = Dense(units=self.encoding_dim, activation="linear")(self.encoded)
        self.log_sigma = Dense(units=self.encoding_dim, activation="linear")(self.encoded)
        z = Lambda(self.sample_z, output_shape=(self.encoding_dim,))([self.mu, self.log_sigma])

        self.decoded_layers_dict = {}
        
        decoder_counter = 0
        
        for i in range(self.decoder_layers):
            if i == 0:
                self.decoded_layers_dict[decoder_counter] = BatchNormalization()
                decoder_counter += 1
                self.decoded_layers_dict[decoder_counter] = Dense(units=self.n_hidden_units, activation="elu")
                decoder_counter += 1
                self.decoded_layers_dict[decoder_counter] = Dropout(rate=0.5)

                self.decoded = self.decoded_layers_dict[decoder_counter - 2](z)
                self.decoded = self.decoded_layers_dict[decoder_counter - 1](self.decoded)
                self.decoded = self.decoded_layers_dict[decoder_counter](self.decoded)

                decoder_counter += 1
            elif i > 0 and i < self.decoder_layers - 1:
                self.decoded_layers_dict[decoder_counter] = BatchNormalization()
                decoder_counter += 1
                self.decoded_layers_dict[decoder_counter] = Dense(units=self.n_hidden_units, activation="elu")
                decoder_counter += 1
                self.decoded_layers_dict[decoder_counter] = Dropout(rate=0.5)

                self.decoded = self.decoded_layers_dict[decoder_counter - 2](self.decoded)
                self.decoded = self.decoded_layers_dict[decoder_counter - 1](self.decoded)
                self.decoded = self.decoded_layers_dict[decoder_counter](self.decoded)

                decoder_counter += 1
            elif i == self.decoder_layers - 1:
                self.decoded_layers_dict[decoder_counter] = BatchNormalization()
                decoder_counter += 1
                self.decoded_layers_dict[decoder_counter] = Dense(units=self.n_hidden_units, activation="elu")

                self.decoded = self.decoded_layers_dict[decoder_counter - 1](self.decoded)
                self.decoded = self.decoded_layers_dict[decoder_counter](self.decoded)
                decoder_counter += 1
        
        # Output would have shape: (batch_size, n_feat).
        self.decoded_layers_dict[decoder_counter] = Dense(units=self.n_feat, activation="sigmoid")
        self.decoded = self.decoded_layers_dict[decoder_counter](self.decoded)

        self.autoencoder = Model(self.input_data, self.decoded)
        self.autoencoder.compile(optimizer=keras.optimizers.Adam(),
                                 loss=self.vae_loss)
            
    def fit(self,
            X,
            y=None):
        self.autoencoder.fit(X if self.denoising is None else X + self.denoising, X,
                             validation_split=0.3,
                             epochs=self.n_epoch,
                             batch_size=self.batch_size,
                             shuffle=True,
                             callbacks=self.callbacks_list, 
                             verbose=1)

        self.encoder = Model(self.input_data, self.mu)

        self.generator_input = Input(shape=(self.encoding_dim,))
        self.generator_output = None
        decoder_counter = 0
            
        for i in range(self.decoder_layers):
            if i == 0:
                self.generator_output = self.decoded_layers_dict[decoder_counter](self.generator_input)
                decoder_counter += 1
                self.generator_output = self.decoded_layers_dict[decoder_counter](self.generator_output)
                decoder_counter += 1
                self.generator_output = self.decoded_layers_dict[decoder_counter](self.generator_output)
                decoder_counter += 1
            elif i > 0 and i < self.decoder_layers - 1:
                self.generator_output = self.decoded_layers_dict[decoder_counter](self.generator_output)
                decoder_counter += 1
                self.generator_output = self.decoded_layers_dict[decoder_counter](self.generator_output)
                decoder_counter += 1
                self.generator_output = self.decoded_layers_dict[decoder_counter](self.generator_output)
                decoder_counter += 1
            elif i == self.decoder_layers - 1:
                self.generator_output = self.decoded_layers_dict[decoder_counter](self.generator_output)
                decoder_counter += 1
                self.generator_output = self.decoded_layers_dict[decoder_counter](self.generator_output)
                decoder_counter += 1

        self.generator_output = self.decoded_layers_dict[decoder_counter](self.generator_output)

        self.generator = Model(self.generator_input, self.generator_output)
                
        return self
    
    def transform(self,
                  X):
        return self.encoder.predict(X)
    
    def sample_z(self,
                 args):
        mu_, log_sigma_ = args
        eps = keras.backend.random_normal(shape=(keras.backend.shape(mu_)[0], self.encoding_dim),
                                          mean=0.0,
                                          stddev=1.0)
        out = mu_ + keras.backend.exp(log_sigma_ / 2) * eps
            
        return out
    
    def vae_loss(self,
                 y_true,
                 y_pred):
        recon = keras.backend.sum(x=keras.backend.square(y_pred - y_true))
        kl = -0.5 * keras.backend.sum(x=1.0 + self.log_sigma - keras.backend.exp(self.log_sigma) - keras.backend.square(self.mu))
        return recon + kl

In [ ]:
import os
import math
import sys
import importlib

import numpy as np

import pandas as pd

from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer, RobustScaler, StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from scipy.stats import norm

import keras
from keras import backend as bkend
from keras.datasets import cifar10, mnist
from keras.layers import Dense, BatchNormalization, Dropout, Flatten, convolutional, pooling
from keras import metrics

# import keras.backend.tensorflow_backend as KTF
# KTF.set_session(get_session(gpu_fraction=0.75, allow_soft_placement=True, log_device_placement=False))

import tensorflow as tf
from tensorflow.python.client import device_lib

from plotnine import *

import matplotlib.pyplot as plt

%matplotlib inline

np.set_printoptions(suppress=True)

os.environ["KERAS_BACKEND"] = "tensorflow"
importlib.reload(bkend)

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3354497126807408506
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10616564208856510467
physical_device_desc: "device: XLA_CPU device"
]


In [ ]:
mnist = mnist.load_data()
(X_train, y_train), (X_test, y_test) = mnist
X_train = np.reshape(X_train, [X_train.shape[0], X_train.shape[1] * X_train.shape[1]])
X_test = np.reshape(X_test, [X_test.shape[0], X_test.shape[1] * X_test.shape[1]])
y_train = y_train.ravel()
y_test = y_test.ravel()
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255.0
X_test /= 255.0

11493376/11490434 [==============================] - 0s 0us/step


#Scikit-learn
We will use the Python machine learning library scikit-learn for data transformation and the classification task. Note that we will code the autoencoders as scikit-learn transformers such that they can be readily used by scikit-learn pipelines.

In [ ]:
scaler_classifier = MinMaxScaler(feature_range=(0.0, 1.0))
logistic = linear_model.LogisticRegression(random_state=666)
linear_mod = linear_model.ElasticNetCV()
lb = LabelBinarizer()
lb = lb.fit(y_train.reshape(y_train.shape[0], 1))

#MNIST: No Autoencoders
We run the MNIST dataset without using an autoencoder. The 2 dimensional tensor of pixel intensities per image for MNIST images are of dimension $\mathbb{R}^{28 \times 28}$. We reshape them as a 1 dimensional tensor of dimension $\mathbb{R}^{784}$ per image. Therefore we have 784, i.e., $28 \times 28 = 784$, features for this task per image.

In [ ]:
pipe_base = Pipeline(steps=[("scaler_classifier", scaler_classifier),
                            ("classifier", logistic)])
pipe_base = pipe_base.fit(X_train, y_train)

acc_base = pipe_base.score(X_test, y_test)

print("The accuracy score for the MNIST classification task without autoencoders: %.6f%%." % (acc_base * 100))

The accuracy score for the MNIST classification task without autoencoders: 92.590000%.


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


#MNIST: PCA
We use a PCA filter that picks the number of components that explain $99\%$ of the variation.


In [ ]:

pipe_pca = Pipeline(steps=[("PCA", PCA(n_components=0.99)),
                           ("scaler_classifier", scaler_classifier),
                           ("classifier", logistic)])
pipe_pca = pipe_base.fit(X_train, y_train)

acc_pca = pipe_pca.score(X_test, y_test)

print("The accuracy score for the MNIST classification task with PCA: %.6f%%." % (acc_pca * 100))

The accuracy score for the MNIST classification task with PCA: 92.590000%.


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


#MNIST: Vanilla Autoencoders
An autoencoder is an unsupervised learning technique where the objective is to learn a set of features that can be used to reconstruct the input data.

Our input data is $X \in \mathbb{R}^{N \times 784}$. An encoder function $E$ maps this to a set of $K$ features such that $E: \mathbb{R}^{N \times 784} \rightarrow \mathbb{R}^{N \times K}$. A decoder function $D$ uses the set of $K$ features to reconstruct the input data such that $D: \mathbb{R}^{N \times K} \rightarrow \mathbb{R}^{N \times 784}$.

$$\begin{align*}
&amp;X \in \mathbb{R}^{N \times 784} \\
&amp;E: \mathbb{R}^{N \times 784} \rightarrow \mathbb{R}^{N \times K} \\
&amp;D: \mathbb{R}^{N \times K} \rightarrow \mathbb{R}^{N \times 784}
\end{align*}$$
Lets denote the reconstructed data as $\tilde{X} = D(E(X))$. The goal is to learn the encoding and decoding functions such that we minimize the difference between the input data and the reconstructed data. An example for an objective function for this task can be the Mean Squared Error (MSE) such that $\frac{1}{N}||\tilde{X} - X||^{2}_{2}$.

We learn the encoding and decoding functions by minimizing the MSE using the parameters that define the encoding and decoding functions: The gradient of the MSE with respect to the parameters are calculated using the chain rule, i.e., backpropagation, and used to update the parameters via an optimization algorithm such as Stochastic Gradient Descent (SGD).

Lets assume we have a single layer autoencoder using the Exponential Linear Unit (ELU) activation function, batch normalization, dropout and the Adaptive Moment (Adam) optimization algorithm. $B$ is the batch size, $K$ is the number of features.

Exponential Linear Unit: The activation function is smooth everywhere and avoids the vanishing gradient problem as the output takes on negative values when the input is negative. $\alpha$ is taken to be $1.0$.
$$\begin{align*}
H_{\alpha}(z) &amp;= 
\begin{cases}
&amp;\alpha\left(\exp(z) - 1\right) \quad \text{if} \quad z &lt; 0 \\
&amp;z \quad \text{if} \quad z \geq 0
\end{cases} \\
\frac{dH_{\alpha}(z)}{dz} &amp;= 
\begin{cases}
&amp;\alpha\left(\exp(z)\right) \quad \text{if} \quad z &lt; 0 \\
&amp;1 \quad \text{if} \quad z \geq 0
\end{cases} 
\end{align*}$$
Batch Normalization: The idea is to transform the inputs into a hidden layer's activation functions. We standardize or normalize first using the mean and variance parameters on a per feature basis and then learn a set of scaling and shifting parameters on a per feature basis that transforms the data. The following equations describe this layer succintly: The parameters we learn in this layer are $\left(\mu_{j}, \sigma_{j}^2, \beta_{j}, \gamma_{j}\right) \quad \forall j \in \{1, \dots, K\}$.
$$\begin{align*}
\mu_{j} &amp;= \frac{1}{B} \sum_{i=1}^{B} X_{i,j} \quad &amp;\forall j \in \{1, \dots, K\} \\
\sigma_{j}^2 &amp;= \frac{1}{B} \sum_{i=1}^{B} \left(X_{i,j} - \mu_{j}\right)^2 \quad &amp;\forall j \in \{1, \dots, K\} \\
\hat{X}_{:,j} &amp;= \frac{X_{:,j} - \mu_{j}}{\sqrt{\sigma_{j}^2 + \epsilon}} \quad &amp;\forall j \in \{1, \dots, K\} \\
Z_{:,j} &amp;= \gamma_{j}\hat{X}_{:,j} + \beta_{j} \quad &amp;\forall j \in \{1, \dots, K\}
\end{align*}$$
Dropout: This regularization technique simply drops the outputs from input and hidden units with a certain probability say $50\%$.

Adam Optimization Algorithm: This adaptive algorithm combines ideas from the Momentum and RMSProp optimization algorithms. The goal is to have some memory of past gradients which can guide future parameters updates. The following equations for the algorithm succintly describe this method assuming $\theta$ is our set of parameters to be learnt and $\eta$ is the learning rate.

$$\begin{align*}
m &amp;\leftarrow \beta_{1}m + \left[\left(1 - \beta_{1}\right)\left(\nabla_{\theta}\text{MSE}\right)\right] \\
s &amp;\leftarrow \beta_{2}s + \left[\left(1 - \beta_{2}\right)\left(\nabla_{\theta}\text{MSE} \otimes \nabla_{\theta}\text{MSE} \right)\right] \\
\theta &amp;\leftarrow \theta - \eta m \oslash \sqrt{s + \epsilon}
\end{align*}$$

In [ ]:
autoencoder = VanillaAutoencoder(n_feat=X_train.shape[1],
                                 n_epoch=50,
                                 batch_size=100,
                                 encoder_layers=3,
                                 decoder_layers=3,
                                 n_hidden_units=1000,
                                 encoding_dim=500,
                                 denoising=None)

print(autoencoder.autoencoder.summary())

pipe_autoencoder = Pipeline(steps=[("autoencoder", autoencoder),
                                   ("scaler_classifier", scaler_classifier),
                                   ("classifier", logistic)])

pipe_autoencoder = pipe_autoencoder.fit(X_train, y_train)

acc_autoencoder = pipe_autoencoder.score(X_test, y_test)

print("The accuracy score for the MNIST classification task with an autoencoder: %.6f%%." % (acc_autoencoder * 100))

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 1000)              785000    
_________________________________________________________________
dropout (Dropout)            (None, 1000)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1000)              4000      
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)             

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


The accuracy score for the MNIST classification task with an autoencoder: 96.610000%.


#MNIST: Denoising Autoencoders
The idea here is to add some noise to the data and try to learn a set of robust features that can reconstruct the non-noisy data from the noisy data. The MSE objective functions is as follows, $\frac{1}{N}||D(E(X + \epsilon)) - X||^{2}_{2}$, where $\epsilon$ is some noise term.

$$\begin{align*}
&amp;X \in \mathbb{R}^{N \times 784} \\
&amp;E: \mathbb{R}^{N \times 784} \rightarrow \mathbb{R}^{N \times K} \\
&amp;D: \mathbb{R}^{N \times K} \rightarrow \mathbb{R}^{N \times 784}
\end{align*}$$

In [ ]:
noise = 0.10 * np.reshape(np.random.uniform(low=0.0, 
                                            high=1.0, 
                                            size=X_train.shape[0] * X_train.shape[1]), 
                          [X_train.shape[0], X_train.shape[1]])

denoising_autoencoder = VanillaAutoencoder(n_feat=X_train.shape[1],
                                           n_epoch=50,
                                           batch_size=100,
                                           encoder_layers=3,
                                           decoder_layers=3,
                                           n_hidden_units=1000,
                                           encoding_dim=500,
                                           denoising=noise)

print(denoising_autoencoder.autoencoder.summary())

pipe_denoising_autoencoder = Pipeline(steps=[("autoencoder", denoising_autoencoder),
                                             ("scaler_classifier", scaler_classifier),
                                             ("classifier", logistic)])

pipe_denoising_autoencoder = pipe_denoising_autoencoder.fit(X_train, y_train)

acc_denoising_autoencoder = pipe_denoising_autoencoder.score(X_test, y_test)

print("The accuracy score for the MNIST classification task with a denoising autoencoder: %.6f%%." % (acc_denoising_autoencoder * 100))

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 784)               3136      
_________________________________________________________________
dense_8 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dropout_4 (Dropout)          (None, 1000)              0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 1000)              4000      
_________________________________________________________________
dense_9 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000)             

#MNIST: Sequence to Sequence Autoencoders
Given our mortgage default example a potentially more useful deep learning architecture might be the Recurrent Neural Network (RNN), specifically their state of the art variant the Long Short Term Memory (LSTM) network. The goal is to explicitly take into account the sequential nature of the raw data.

$$\begin{align*}
&amp;X \in \mathbb{R}^{N \times 28 \times 28} \\
&amp;E: \mathbb{R}^{N \times 28 \times 28} \rightarrow \mathbb{R}^{N \times K} \\
&amp;D: \mathbb{R}^{N \times K} \rightarrow \mathbb{R}^{N \times 28 \times 28}
\end{align*}$$
The gradients in a RNN depend on the parameter matrices defined for the model. Simply put these parameter matrices can end up being multiplied many times over and hence cause two major problems for learning: Exploding and vanishing gradients. If the spectral radius of the parameter matrices, i.e., the maximum absolute value of the eigenvalues of a matrix, is more than 1 then gradients can become large enough, i.e., explode in value, such that learning diverges and similarly if the spectral radius is less than 1 then gradients can become small, i.e., vanish in value, such that the next best transition for the parameters cannot be reliably calculated. Appropriate calculation of the gradient is important for estimating the optimal set of parameters that define a machine learning method and the LSTM network overcomes these problems in a vanilla RNN. We now define the LSTM network for 1 time step, i.e., 1 memory cell.

We calculate the value of the input gate, the value of the memory cell state at time period $t$ where $f(x)$ is some activation function and the value of the forget gate:

$$\begin{align*}
i_{t} &amp;= \sigma(W_{i}x_{t} + U_{i}h_{t-1} + b_{i}) \\
\tilde{c_{t}} &amp;= f(W_{c}x_{t} + U_{c}h_{t-1} + b_{c}) \\
f_{t} &amp;= \sigma(W_{f}x_{t} + U_{f}h_{t-1} + b_{f})
\end{align*}$$
The forget gate controls the amount the LSTM remembers, i.e., the value of the memory cell state at time period $t-1$ where $\otimes$ is the hadamard product:

$$\begin{align*}
c_{t} = i_{t} \otimes \tilde{c_{t}} + f_{t} \otimes c_{t-1} 
\end{align*}$$
With the updated state of the memory cell we calculate the value of the outputs gate and finally the output value itself:

$$\begin{align*}
o_{t} &amp;= \sigma(W_{o}x_{t} + U_{o}h_{t-1} + b_{o}) \\
h_{t} &amp;= o_{t} \otimes f(c_{t})
\end{align*}$$
We can have a wide variety of LSTM architectures such as the convolutional LSTM where note that we replace the matrix multiplication operators in the input gate, the initial estimate $\tilde{c_{t}}$ of the memory cell state, the forget gate and the output gate by the convolution operator $*$:

$$\begin{align*}
i_{t} &amp;= \sigma(W_{i} * x_{t} + U_{i} * h_{t-1} + b_{i}) \\
\tilde{c_{t}} &amp;= f(W_{c} * x_{t} + U_{c} * h_{t-1} + b_{c}) \\
f_{t} &amp;= \sigma(W_{f} * x_{t} + U_{f} * h_{t-1} + b_{f}) \\
c_{t} &amp;= i_{t} \otimes \tilde{c_{t}} + f_{t} \otimes c_{t-1} \\ 
o_{t} &amp;= \sigma(W_{o} * x_{t} + U_{o} * h_{t-1} + b_{o}) \\
h_{t} &amp;= o_{t} \otimes f(c_{t})
\end{align*}$$
Another popular variant is the peephole LSTM where the gates are allowed to peep at the memory cell state:

$$\begin{align*}
i_{t} &amp;= \sigma(W_{i}x_{t} + U_{i}h_{t-1} + V_{i}c_{t-1} + b_{i}) \\
\tilde{c_{t}} &amp;= f(W_{c}x_{t} + U_{c}h_{t-1} + V_{c}c_{t-1} + b_{c}) \\
f_{t} &amp;= \sigma(W_{f}x_{t} + U_{f}h_{t-1} + V_{f}c_{t-1} + b_{f}) \\
c_{t} &amp;= i_{t} \otimes \tilde{c_{t}} + f_{t} \otimes c_{t-1} \\ 
o_{t} &amp;= \sigma(W_{o}x_{t} + U_{o}h_{t-1} + V_{o}c_{t} + b_{o}) \\
h_{t} &amp;= o_{t} \otimes f(c_{t})
\end{align*}$$
The goal for the sequence to sequence autoencoder is to create a representation of the raw data using a LSTM as an encoder. This representation will be a sequence of vectors say, $h_{1}, \dots, h_{T}$, learnt from a sequence of raw data vectors say, $x_{1}, \dots, x_{T}$. The final vector of the representation, $h_{T}$, is our encoded representation, also called a context vector. This context vector is repeated as many times as the length of the sequence such that it can be used as an input to a decoder which is yet another LSTM. The decoder LSTM will use this context vector to recontruct the sequence of raw data vectors, $\tilde{x_{1}}, \dots, \tilde{x_{T}}$. If the context vector is useful in the recontruction task then it can be further used for other tasks such as predicting default risk as given in our example.

In [ ]:

seq2seq_autoencoder = Seq2SeqAutoencoder(input_shape=(int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))),
                                         n_epoch=50,
                                         batch_size=100,
                                         encoder_layers=3,
                                         decoder_layers=3,
                                         n_hidden_units=200,
                                         encoding_dim=200,
                                         stateful=False,
                                         denoising=None)

print(seq2seq_autoencoder.autoencoder.summary())

pipe_seq2seq_autoencoder = Pipeline(steps=[("autoencoder", seq2seq_autoencoder),
                                           ("scaler_classifier", scaler_classifier),
                                           ("classifier", logistic)])

pipe_seq2seq_autoencoder = pipe_seq2seq_autoencoder.fit(np.reshape(X_train, [X_train.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))]),
                                                        y_train)

acc_seq2seq_autoencoder = pipe_seq2seq_autoencoder.score(np.reshape(X_test, [X_test.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))]), y_test)

print("The accuracy score for the MNIST classification task with a sequence to sequence autoencoder: %.6f%%." % (acc_seq2seq_autoencoder * 100))


#MNIST: Variational Autoencoders
We now combine Bayesian inference with deep learning by using variational inference to train a vanilla autoencoder. This moves us towards generative modelling which can have further use cases in semi-supervised learning. The other benefit of training using Bayesian inference is that we can be more robust to higher capacity deep learners, i.e., avoid overfitting.

$$\begin{align*}
&amp;X \in \mathbb{R}^{N \times 784} \\
&amp;E: \mathbb{R}^{N \times 784} \rightarrow \mathbb{R}^{N \times K} \\
&amp;D: \mathbb{R}^{N \times K} \rightarrow \mathbb{R}^{N \times 784}
\end{align*}$$
Assume $X$ is our raw data while $Z$ is our learnt representation.
We have a prior belief on our learnt representation:
$$\begin{align*}
p(Z)
\end{align*}$$
The posterior distribution for our learnt representation is:
$$\begin{align*}
p(Z|X)=\frac{p(X|Z)p(Z)}{p(X)}
\end{align*}$$
The marginal likelihood, $p(X)$, is often intractable causing the posterior distribution, $p(Z|X)$, to be intractable:
$$\begin{align*}
p(X)=\int_{Z}p(X|Z)p(Z)dZ
\end{align*}$$
We therefore need an approximate posterior distribution via variational inference that can deal with the intractability. This additionally also provides the benefit of dealing with large scale datasets as generally Markov Chain Monte Carlo (MCMC) methods are not well suited for large scale datasets. One might also consider Laplace approximation for the approximate posterior distribution however we will stick with variational inference as it allows a richer set of approximations compared to Laplace approximation. Laplace approximation simply amounts to finding the Maximum A Posteriori (MAP) estimate to an augmented likelihood optimization, taking the negative of the inverse of the Hessian at the MAP estimate to estimate the variance-covariance matrix and finally use the variance-covariance matrix with a multivariate Gaussian distribution or some other appropriate multivariate distribution.

Assume that our approximate posterior distribution, which is also our probabilistic encoder, is given as:

$$\begin{align*}
q(Z|X)
\end{align*}$$
Our probabilistic decoder is given by:
$$\begin{align*}
p(X|Z)
\end{align*}$$
Given our setup above with regards to an encoder and a decoder let us now write down the optimization problem where $\theta$ are the generative model parameters while $\phi$ are the variational parameters:
$$\begin{align*}
\log{p(X)}= \underbrace{D_{KL}(q(Z|X)||p(Z|X))}_\text{Intractable as p(Z|X) is intractable} + \underbrace{\mathcal{L}(\theta, \phi|X)}_\text{Evidence Lower Bound or ELBO}
\end{align*}$$
Note that $D_{KL}(q(Z|X)||p(Z|X))$ is non-negative therefore that makes the ELBO a lower bound on $\log{p(X)}$:
$$\begin{align*}
\log{p(X)}\geq \mathcal{L}(\theta, \phi|X) \quad \text{as} \quad D_{KL}(q(Z|X)||p(Z|X)) \geq 0
\end{align*}$$
Therefore we can alter our optimization problem to look only at the ELBO:
$$\begin{align*}
\mathcal{L}(\theta, \phi|X) &amp;= \mathbb{E}_{q(Z|X)}\left[\log{p(X,Z)} - \log{q(Z|X)}\right] \\
&amp;= \mathbb{E}_{q(Z|X)}\left[\underbrace{\log{p(X|Z)}}_\text{Reconstruction error} + \log{p(Z)} - \log{q(Z|X)}\right] \\
&amp;= \mathbb{E}_{q(Z|X)}\left[\underbrace{\log{p(X|Z)}}_\text{Reconstruction error} - \underbrace{D_{KL}(q(Z|X)||p(Z))}_\text{Regularization}\right] \\
&amp;= \int_{Z} \left[\log{p(X|Z)} - D_{KL}(q(Z|X)||p(Z))\right] q(Z|X) dZ
\end{align*}$$
The above integration problem can be solved via Monte Carlo integration as $D_{KL}(q(Z|X)||p(Z))$ is not intractable. Assuming that the probabilistic encoder $q(Z|X)$ is a multivariate Gaussian with a diagonal variance-covariance matrix we use the reparameterization trick to sample from this distribution say $M$ times in order to calculate the expectation term in the ELBO optimization problem. The reparameterization trick in this particular case amounts to sampling $M$ times from the standard Gaussian distribution, multiplying the samples by $\sigma$ and adding $\mu$ to the samples.

$\mu$ is our learnt representation used for the reconstruction of the raw data. If the learnt representation is useful it can then be used for other tasks as well.

This is a powerful manner of combining Bayesian inference with deep learning. Variational inference used in this manner can be applied to various deep learning architectures and has further links with the Generative Adversarial Network (GAN). We explore the use of adversarial learning in representation learning in another repo/paper.

In [ ]:
encoding_dim = 500

variational_autoencoder = VariationalAutoencoder(n_feat=X_train.shape[1],
                                                 n_epoch=50,
                                                 batch_size=100,
                                                 encoder_layers=3,
                                                 decoder_layers=3,
                                                 n_hidden_units=1000,
                                                 encoding_dim=encoding_dim,
                                                 denoising=None)

print(variational_autoencoder.autoencoder.summary())

pipe_variational_autoencoder = Pipeline(steps=[("autoencoder", variational_autoencoder),
                                               ("scaler_classifier", scaler_classifier),
                                               ("classifier", logistic)])

pipe_variational_autoencoder = pipe_variational_autoencoder.fit(X_train, y_train)

acc_variational_autoencoder = pipe_variational_autoencoder.score(X_test, y_test)

print("The accuracy score for the MNIST classification task with a variational autoencoder: %.6f%%." % (acc_variational_autoencoder * 100))

if encoding_dim == 2:
    test_encoded_df = pd.DataFrame(pipe_variational_autoencoder.named_steps["autoencoder"].encoder.predict(X_test))
    test_encoded_df["Target"] = y_test
    test_encoded_df.columns.values[0:2] = ["Encoding_1", "Encoding_2"]

    scaler_plot = MinMaxScaler(feature_range=(0.25, 0.75))
    scaler_plot = scaler_plot.fit(test_encoded_df[["Encoding_1", "Encoding_2"]])
    test_encoded_df[["Encoding_1", "Encoding_2"]] = scaler_plot.transform(test_encoded_df[["Encoding_1", "Encoding_2"]])

    cluster_plot = ggplot(test_encoded_df) + \
    geom_point(aes(x="Encoding_1", 
                   y="Encoding_2", 
                   fill="factor(Target)"),
               size=1,
               color = "black") + \
    xlab("Encoding dimension 1") + \
    ylab("Encoding dimension 2") + \
    ggtitle("Variational autoencoder with 2-dimensional encoding") + \
    theme_matplotlib()
    print(cluster_plot)

    n = 30
    digit_size = 28
    figure = np.zeros((digit_size * n, digit_size * n))
    grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
    grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

    for i, xi in enumerate(grid_x):
        for j, yi in enumerate(grid_y):
            z_sample = np.array([[xi, yi]])
            x_decoded = pipe_variational_autoencoder.named_steps["autoencoder"].generator.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[i * digit_size: (i + 1) * digit_size, j * digit_size: (j + 1) * digit_size] = digit

    plt.figure(figsize=(20, 20))
    plt.imshow(figure, cmap="Greys_r")
    plt.title("Variational Autoencoder (VAE) with 2-dimensional encoding\nGenerating new images")
    plt.xlabel("Encoding dimension 1")
    plt.ylabel("Encoding dimension 2")
    plt.savefig(fname="VAE_Generated_Images.png")
    plt.show()

#MNIST: 2 Dimensional Convolutional Autoencoders
For 2 dimensional convolution filters the idea is similar as for the 1 dimensional convolution filters. We will stick to our previously mentioned banking example to illustrate this point.

$$\begin{align*}
x = 
\begin{array}
{l}
\text{Period 1} \\ \text{Period 2} \\ \text{Period 3} \\ \text{Period 4} \\ \text{Period 5}
\end{array}
    \left[
    \begin{array}
    {ccc}
    \$0 &amp; \$0 &amp; \$0 \\
    \$0 &amp; \$200 &amp; \$0 \\
    \$100 &amp; \$0 &amp; \$0 \\
    \$0 &amp; \$0 &amp; \$300 \\
    \$0 &amp; \$0 &amp; \$0
    \end{array}
    \right]
\end{align*}$$
In the 2 dimensional tensor of raw transactions data now we have 5 historical time periods, i.e., the rows, and 3 different transaction types, i.e., the columns. We will use a kernel, $\alpha \in \mathbb{R}^{2\times3}$, to extract useful features from the raw data. The choice of such a kernel means that we are interested in finding a feature map across all 3 transaction types and 2 historical time periods. We will use a stride length of 1 and a valid convolution to extract features over different patches of the raw data. The following will illustrate this point where $x_{\text{patch}} \subset x$:

$$\begin{align*}
\alpha &amp;=
    \left[
    \begin{array}
    {ccc}
    \alpha_{1,1} &amp; \alpha_{1,2} &amp; \alpha_{1,3} \\
    \alpha_{2,1} &amp; \alpha_{2,2} &amp; \alpha_{2,3}
    \end{array}
    \right] \\
x_{\text{patch}} &amp;= 
    \left[
    \begin{array}
    {ccc}
    \$0 &amp; \$0 &amp; \$0 \\
    \$0 &amp; \$200 &amp; \$0
    \end{array}
    \right] \\
\mathbf{C}(x=x_{\text{patch}}|\alpha) &amp;= x * \alpha \\
&amp;= \sum_{t=1}^{2} \sum_{k=1}^{3} x_{t,k} \alpha_{t,k}
\end{align*}$$
The principles and ideas apply to 2 dimensional convolution filters as they do for their 1 dimensional counterparts there we will not repeat them here.

$$\begin{align*}
&amp;X \in \mathbb{R}^{N \times 28 \times 28} \\
&amp;E: \mathbb{R}^{N \times 28 \times 28} \rightarrow \mathbb{R}^{N \times K} \\
&amp;D: \mathbb{R}^{N \times K} \rightarrow \mathbb{R}^{N \times 28 \times 28}
\end{align*}$$

In [ ]:

convolutional2D_autoencoder = Convolutional2DAutoencoder(input_shape=(int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5)), 1),
                                                         n_epoch=5,
                                                         batch_size=100,
                                                         encoder_layers=3,
                                                         decoder_layers=3,
                                                         filters=100,
                                                         kernel_size=(8, 8),
                                                         strides=(1, 1),
                                                         pool_size=(4, 4),
                                                         denoising=None)

print(convolutional2D_autoencoder.autoencoder.summary())

pipe_convolutional2D_autoencoder = Pipeline(steps=[("autoencoder", convolutional2D_autoencoder),
                                                   ("scaler_classifier", scaler_classifier),
                                                   ("classifier", logistic)])

pipe_convolutional2D_autoencoder = pipe_convolutional2D_autoencoder.fit(np.reshape(X_train, [X_train.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5)), 1]),
                                                                        y_train)

acc_convolutional2D_autoencoder = pipe_convolutional2D_autoencoder.score(np.reshape(X_test, [X_test.shape[0], int(math.pow(X_test.shape[1], 0.5)), int(math.pow(X_test.shape[1], 0.5)), 1]), y_test)

print("The accuracy score for the MNIST classification task with a 2 dimensional convolutional autoencoder: %.6f%%." % (acc_convolutional2D_autoencoder * 100))